<a href="https://colab.research.google.com/github/p4zaa/OTTO-Multi-Objective-Recommender-System/blob/main/%5BDEVELOP1_6%5D_OTTO_%E2%80%93_Multi_Objective_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version Logs
* [View in my Github](https://github.com/p4zaa/OTTO-Multi-Objective-Recommender-System)

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

In [54]:
import pandas as pd, numpy as np
from itertools import product
import io, os, json
import tqdm

import time

from sklearn.metrics import mean_squared_error

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import to_hetero
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, GraphConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import HGTLoader, NeighborLoader, LinkNeighborLoader

!pip install -q sentence-transformers

%matplotlib inline

In [4]:
!pip3 install -q torch torchvision torchaudio
!pip install -U polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import polars as pl

In [6]:
print('Pytorch CUDA Version is ', torch.version.cuda)

Pytorch CUDA Version is  11.6


In [7]:
!nvidia-smi

Mon Jan  2 17:00:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
torch.cuda.is_available()

True

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Competition Dataset

In [10]:

from google.colab import files
files.upload() # expire any previous token(s) and upload recreated token
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                           title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                     COVID-19 Dataset                                     5MB  2022-11-13 15:47:17          14942        423  1.0              
devrimtuner/list-of-moststreamed-songs-on-spotify             Top 100 Spotify Songs👑🎤🎧🎼                            3KB  2022-12-30 05:42:54            319         26  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india  Credit Card Spending Habits in India               319KB  2022-12-14 07:30:37           1513         51  1.0              
die9origephit/fifa-world-cup-2022-complet

In [11]:
!kaggle datasets download 'radek1/otto-full-optimized-memory-footprint' -p /content/kaggle/ --unzip

100% 1.09G/1.09G [01:00<00:00, 19.8MB/s]
100% 1.09G/1.09G [01:00<00:00, 19.4MB/s]


## Files
- **train.jsonl** - the training data, which contains full session data
  * `session` - the unique session id
  * `events` - the time ordered sequence of events in the session
    * `aid` - the article id (product code) of the associated event
    * `ts` - the Unix timestamp of the event
    * `type` - the event type, i.e., whether a product was clicked, added to the user's cart, or ordered during the session
###### {'clicks': 0, 'carts': 1, 'orders': 2}
- **test.jsonl** - the test data, which contains truncated session data
  * your task is to predict the next `aid` clicked after the session truncation, as well as the the remaining `aids` that are added to `carts` and `orders`; you may predict up to 20 values for each session `type`
- **sample_submission.csv** - a sample submission file in the correct format

In [12]:
# Pandas
#df = pd.read_parquet('/content/kaggle/train.parquet')

# Polars
df = pl.read_parquet('/content/kaggle/train.parquet')

In [13]:
# sample 10%(frac=0.1) of data
#df = df.sample(frac=0.01, replace=False)

# Polars
df = df.sample(frac=0.001)

In [14]:
df = df.to_pandas()

In [15]:
df

,session,aid,ts,type
0,2359067,1436622,1660673310,0
1,789548,56365,1660145805,0
2,12683589,458164,1661678902,0
3,510558,92307,1660483127,0
4,569378,989193,1660199424,0
...,...,...,...,...
216711,12469469,819288,1661655284,0
216712,68877,125973,1660916085,2
216713,7088532,1138171,1660250938,0
216714,10335524,1188390,1661330988,0


In [16]:
#df.isna().sum()

In [17]:
df['type'].unique()

array([0, 2, 1], dtype=uint8)

In [18]:
#df.info()

# Construct Graph Data (memory-efficient optimized)

### `edge_index` new code construct

In [19]:
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}

In [20]:
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

<ipython-input-20-fdbbf2e7eb54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['session'] = connected['session'].map(source_idx)
<ipython-input-20-fdbbf2e7eb54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['aid'] = connected['aid'].map(target_idx)


In [21]:
source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values))

<ipython-input-21-23961f25841f>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  edge_index = torch.tensor((source.values, target.values))


### Nodes and Edges Attribute

In [22]:
df['session']

0          2359067
1           789548
2         12683589
3           510558
4           569378
            ...   
216711    12469469
216712       68877
216713     7088532
216714    10335524
216715     4570654
Name: session, Length: 216716, dtype: int32

In [23]:
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
aid_features = torch.rand((aid_num_nodes, 32)) # Create (random) article features with shape [num_node_aid, dimensions]

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)

In [24]:
edge_label

tensor([0, 0, 0,  ..., 0, 0, 0])

In [25]:
edge_index

tensor([[     0,      1,      2,  ..., 208337, 208338, 208339],
        [     0,      1,      2,  ..., 127650,   4738,  22900]])

In [26]:
aid_features.shape

torch.Size([127651, 32])

In [27]:
edge_label.shape

torch.Size([216716])

### Construct HeteroData

In [28]:
node_types = {
    'session': {
        #'num_nodes': session_num_nodes,
        'node_id': torch.tensor(list(source_idx.values()), dtype=torch.int64)
    },
    'aid': {
        'x': aid_features,
        'node_id': torch.tensor(list(target_idx.values()), dtype=torch.int64)
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}

In [29]:
data = HeteroData({**node_types, **edge_types})

In [30]:
data

HeteroData(
  session={ node_id=[208340] },
  aid={
    x=[127651, 32],
    node_id=[127651]
  },
  (session, event, aid)={
    edge_index=[2, 216716],
    edge_label=[216716]
  }
)

In [31]:
data.metadata()

(['session', 'aid'], [('session', 'event', 'aid')])

In [32]:
node_types, edge_types = data.metadata()
print('Node types:', node_types)
print('Edge types:',edge_types)

Node types: ['session', 'aid']
Edge types: [('session', 'event', 'aid')]


In [33]:
print('Isolated nodes?', data.has_isolated_nodes())
print('Self loops?', data.has_self_loops())
print('Undirected graph? ', data.is_undirected())

Isolated nodes? False
Self loops? False
Undirected graph?  False


# Graph-based Modeling [follow [this](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py) sample]

In [34]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Construct Undirected Graph

In [35]:
# add sesion features for message passing:
data['session'].x = torch.rand(data['session'].num_nodes, 32)

In [36]:
# Add user node features for message passing:
#data['session'].x = torch.eye(data['session'].num_nodes, device=device)
#del data['session'].num_nodes

In [37]:
# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)

In [38]:
del data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.

In [39]:
data

HeteroData(
  session={
    node_id=[208340],
    x=[208340, 32]
  },
  aid={
    x=[127651, 32],
    node_id=[127651]
  },
  (session, event, aid)={
    edge_index=[2, 216716],
    edge_label=[216716]
  },
  (aid, rev_event, session)={ edge_index=[2, 216716] }
)

### Calculate Weight

In [40]:
data['session', 'aid'].edge_label.shape

torch.Size([216716])

In [41]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.

counts = torch.bincount(data['session', 'aid'].edge_label)

# Set weights normalized by (max count/each count)
weight = counts.max() / counts

In [42]:
counts

tensor([194730,  16948,   5038])

In [43]:
weight

tensor([ 1.0000, 11.4899, 38.6522])

In [44]:
data_dict = {'type': (counts, '# rows','coral'), 'weights': (weight, 'weights','royalblue')}

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=np.arange(6), y=counts.detach().cpu().numpy(),
               name = 'nb rows', line_color= 'coral'))
fig.add_trace(
    go.Scatter(x=np.arange(6), y=weight.detach().cpu().numpy(),
               name = 'weights', line_color= 'royalblue'),  secondary_y=True)


fig.update_yaxes(title_text="# rows", secondary_y=False)
fig.update_yaxes(title_text="weights", secondary_y=True)
fig.update_xaxes(title_text="Type")
fig

### [Follow [this sample](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py)] Train/Val/Test Link Level Splits

In [45]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('session', 'event', 'aid')],
    rev_edge_types=[('aid', 'rev_event', 'session')],
)(data)

### Mini Batch

In [46]:
# Define seed edges:
edge_label_index = train_data['session', 'event', 'aid'].edge_label_index
edge_label = train_data['session', 'event', 'aid'].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[5, 10],  # TODO
    neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

#assert sampled_data["session", "event", "aid"].edge_label_index.size(1) == 128
#assert sampled_data["session", "event", "aid"].edge_label.min() == 0
#assert sampled_data["session", "event", "aid"].edge_label.max() == 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[333],
    x=[333, 32]
  },
  aid={
    x=[146, 32],
    node_id=[146]
  },
  (session, event, aid)={
    edge_index=[2, 326],
    edge_label=[128],
    edge_label_index=[2, 128],
    input_id=[128]
  },
  (aid, rev_event, session)={ edge_index=[2, 340] }
)


### [New weight calculation code]

In [47]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.
weight = torch.bincount(train_data['session', 'aid'].edge_label)
weight = weight.max() / weight

In [48]:
weight

tensor([ 1.0000, 11.4500, 38.5380])

### Model and loss functions

#### Loss function

In [49]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

#### Model

In [64]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)

        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        self.aid_lin = Linear(-1, hidden_channels)
        self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = {
            'session': self.session_emb(data['session'].node_id.to(device)),
            'aid': self.aid_lin(data['aid'].x.to(device).float()) + self.aid_emb(data['aid'].node_id.to(device))
        }
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [65]:
model = Model(hidden_channels=32).to(device)

In [62]:
data

HeteroData(
  session={
    node_id=[208340],
    x=[208340, 32]
  },
  aid={
    x=[127651, 32],
    node_id=[127651]
  },
  (session, event, aid)={
    edge_index=[2, 216716],
    edge_label=[216716]
  },
  (aid, rev_event, session)={ edge_index=[2, 216716] }
)

In [66]:
# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:
for epoch in range(0, 1):
  for sampled_data in tqdm.tqdm(train_loader):
    with torch.no_grad():
        model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


100%|██████████| 1355/1355 [00:04<00:00, 336.46it/s]


In [56]:
'''
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)
'''

"\ndef train():\n    model.train()\n    optimizer.zero_grad()\n    pred = model(train_data.x_dict, train_data.edge_index_dict,\n                 train_data['session', 'aid'].edge_label_index)\n    target = train_data['session', 'aid'].edge_label\n    loss = weighted_mse_loss(pred, target, weight)\n    loss.backward()\n    optimizer.step()\n    return float(loss)\n"

In [57]:
'''
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=-1, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)
'''

"\n@torch.no_grad()\ndef test(data):\n    model.eval()\n    pred = model(data.x_dict, data.edge_index_dict,\n                 data['session', 'aid'].edge_label_index)\n    pred = pred.clamp(min=-1, max=2)\n    target = data['session', 'aid'].edge_label.float()\n    rmse = F.mse_loss(pred, target).sqrt()\n    return float(rmse)\n"

In [67]:
def train(train_data=sampled_data):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight.to(device))
    #loss = F.binary_cross_entropy_with_logits(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss), pred, model

In [68]:
@torch.no_grad()
def test(data=data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

### [non-batch] Training Heterogenous Link-level GNN

In [60]:
'''
for epoch in range(0, 30):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')
'''

"\nfor epoch in range(0, 30):\n    loss = train()\n    train_rmse = test(train_data)\n    val_rmse = test(val_data)\n    test_rmse = test(test_data)\n    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '\n          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')\n"

### [mini-batch] Training Heterogenous Link-level GNN

In [69]:
import torch.nn.functional as F

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

#model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        loss, pred, model = train(train_data=sampled_data.to(device))
        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`

        #loss.backward()
        #optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 1355/1355 [00:40<00:00, 33.73it/s]


Epoch: 000, Loss: 1.8490


100%|██████████| 1355/1355 [00:40<00:00, 33.50it/s]


Epoch: 001, Loss: 1.8011


...

Epoch: 007, Loss: 1.7986


100%|██████████| 1355/1355 [00:39<00:00, 34.40it/s]


Epoch: 008, Loss: 1.7994


100%|██████████| 1355/1355 [00:39<00:00, 34.38it/s]

Epoch: 009, Loss: 1.7993


In [70]:
model

Model(
  (encoder): GraphModule(
    (conv1): ModuleDict(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
  )
  (decoder): EdgeDecoder(
    (lin1): Linear(64, 32, bias=True)
    (lin2): Linear(32, 1, bias=True)
  )
  (aid_lin): Linear(32, 32, bias=True)
  (session_emb): Embedding(208340, 32)
  (aid_emb): Embedding(127651, 32)
)

In [71]:
data.x_dict

{'session': tensor([[0.4068, 0.4974, 0.5836,  ..., 0.5933, 0.8149, 0.9005],
         [0.8796, 0.2133, 0.1040,  ..., 0.7070, 0.7537, 0.4335],
         [0.8060, 0.9245, 0.8960,  ..., 0.7505, 0.4845, 0.8024],
         ...,
         [0.5482, 0.7249, 0.5086,  ..., 0.7595, 0.9988, 0.9627],
         [0.2011, 0.8444, 0.7196,  ..., 0.8700, 0.2525, 0.0840],
         [0.6580, 0.4014, 0.1542,  ..., 0.4910, 0.8653, 0.0495]]),
 'aid': tensor([[0.1780, 0.0782, 0.5064,  ..., 0.6507, 0.3055, 0.0411],
         [0.6047, 0.4931, 0.0727,  ..., 0.2601, 0.9701, 0.7170],
         [0.3824, 0.4158, 0.1855,  ..., 0.8392, 0.2156, 0.2997],
         ...,
         [0.1009, 0.7897, 0.4520,  ..., 0.8765, 0.2897, 0.1161],
         [0.1567, 0.8596, 0.9713,  ..., 0.7010, 0.8859, 0.9374],
         [0.0577, 0.2148, 0.6559,  ..., 0.3413, 0.4905, 0.9442]])}

In [72]:
model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

{'session': tensor([[-0.1998,  0.3251,  0.1852,  ...,  0.7644,  0.6157, -0.0876],
         [-0.0547, -0.0925,  0.1507,  ...,  0.3292,  0.5206, -0.1926],
         [-0.3975,  0.0948,  0.1299,  ...,  0.5606,  0.4732, -0.1465],
         ...,
         [ 0.0177,  0.1852,  0.2423,  ...,  0.4145,  0.4012, -0.1082],
         [-0.2985,  0.1386,  0.2573,  ...,  0.8903,  0.7655,  0.0909],
         [-0.0095,  0.0768,  0.0943,  ...,  0.6640,  0.6556, -0.2670]],
        device='cuda:0', grad_fn=<AddBackward0>),
 'aid': tensor([[-1.1049, -0.0045,  1.0334,  ...,  0.3684, -0.1014, -0.6878],
         [-0.5533,  0.2391,  0.3096,  ...,  0.1293, -0.1078, -0.2206],
         [-1.0127,  0.0359,  0.8947,  ...,  0.3901, -0.1826, -0.6484],
         ...,
         [-0.6590,  0.0106,  0.4876,  ..., -0.1731, -0.4963, -0.3757],
         [-0.4970,  0.1395,  0.5390,  ...,  0.2512, -0.5452, -0.4043],
         [-0.2749, -0.0106,  0.4551,  ..., -0.0763, -0.1711, -0.0351]],
        device='cuda:0', grad_fn=<AddBackward0>)}

### Evaluate a Heterogenous Linklevel GNN

In [73]:
# Define the validation seed edges:
edge_label_index = val_data['session', 'event', 'aid'].edge_label_index
edge_label = val_data['session', 'event', 'aid'].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,  # TODO
    num_neighbors=[20, 10],  # TODO
    #neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data['session', 'event', 'aid'].edge_label_index.size(1) == 3 * 128
assert sampled_data['session', 'event', 'aid'].edge_label.min() >= 0
assert sampled_data['session', 'event', 'aid'].edge_label.max() <= 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[1592],
    x=[1592, 32]
  },
  aid={
    x=[472, 32],
    node_id=[472]
  },
  (session, event, aid)={
    edge_index=[2, 1230],
    edge_label=[384],
    edge_label_index=[2, 384],
    input_id=[384]
  },
  (aid, rev_event, session)={ edge_index=[2, 1266] }
)


In [74]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# You need the labels to binarize
labels = [0, 1, 2]

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        _, pred, ground_truth = test(data=sampled_data.to(device))
        preds.append(pred)
        ground_truths.append(ground_truth)
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        #raise NotImplementedError

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Binarize `pred` with shape (n_samples, n_classes)
pred = label_binarize(pred.astype(int), classes=labels)

# Binarize `ground_truth` with shape (n_samples, n_classes)
ground_truth = label_binarize(ground_truth, classes=labels)

#rmse = _
auc = roc_auc_score(ground_truth, pred, multi_class='ovo', average='weighted')
print()
print(f"Validation AUC: {auc:.4f}")
#print('RMSE: ', rmse)

100%|██████████| 57/57 [00:00<00:00, 90.10it/s]


Validation AUC: 0.5000


# Test dataset

In [ ]:
#df = pd.read_parquet('/content/kaggle/test.parquet')

In [ ]:
#df = df.sample(frac=0.01, replace=False)

In [ ]:
#df

## Construct heterogenous graph for test

In [ ]:
'''
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}
'''

In [ ]:
'''
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values)).type(torch.int64)
'''

In [ ]:
'''
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
aid_features = torch.rand((aid_num_nodes, 300)) # Create (random) article features with shape [num_node_aid, dimensions]

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)
'''

In [ ]:
'''
node_types = {
    'session': {
        'num_nodes': session_num_nodes
    },
    'aid': {
        'x': aid_features
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label,
        'edge_label_index': edge_index
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}
'''

In [ ]:
#Rtest_data = HeteroData({**node_types, **edge_types})

In [ ]:
#data

In [ ]:
#Rtest_data

In [ ]:
'''
# add sesion features for message passing:
Rtest_data['session'].x = torch.rand(Rtest_data['session'].num_nodes, 300)

Rtest_data = T.ToUndirected()(Rtest_data)
del Rtest_data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.
'''

In [ ]:
#Rtest_data

## Testing

In [ ]:
'''
with torch.no_grad():
    model.eval()
    gg = model(Rtest_data.x_dict, Rtest_data.edge_index_dict, Rtest_data['session', 'aid'].edge_label_index)
'''

In [ ]:
#data['session', 'aid'].edge_label

In [ ]:
#Rtest_data['session', 'aid'].edge_label

In [ ]:
#gg

In [ ]:
#data

# [TEMPORARY DROP]

In [ ]:
# Temporary comment
'''
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, conv):
        super().__init__()
        # conv(#in_channels, #out_channels)
        ''''''
        in_channels (int or tuple): 
            Size of each input sample, or :obj:`-1` to
            derive the size from the first input(s) to the forward method.
            A tuple corresponds to the sizes of source and target
            dimensionalities.
        ''''''
        self.conv1 = conv((-1, -1), hidden_channels)
        self.conv2 = conv((-1, -1), out_channels)
        self.linear1 = Linear(-1, out_channels)
        self.linear2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x0 = self.linear1(x)
        x2 = self.conv1(x0, edge_index).relu()
        x3 = self.conv2(x2, edge_index)
        x4 = self.linear2(x2 + x3)
        # Add combined layer to reduce over-smoothing
        return x4

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels,  conv=SAGEConv):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels,  conv) # Initialize GNNEncoder
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels) # Initialze EdgeDecoder

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict) # Here the call from model.encoder(...)
        return self.decoder(z_dict, edge_label_index)
'''

In [ ]:
'''
def train(train_data, model, optimizer, loss=weighted_mse_loss):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss.sqrt())
'''

In [ ]:
## set pred.clamp
'''
@torch.no_grad()
def test(data, model, metric=F.mse_loss):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse) # Return RMSE loss
'''

In [ ]:
#from tqdm import tqdm
#from IPython.display import clear_output

In [ ]:
"""
def train_test(model, model_params, learning_rate=0.01, e_patience = 10, min_acc= 0.05, n_epochs=500):
    t0 = time.time()

    model = model(**model_params) # Define the model

    # Due to lazy initialization, we need to run one model step so the number
    # of parameters can be inferred:
    with torch.no_grad():
        model.encoder(train_data.x_dict, train_data.edge_index_dict) # Run once with torch.no_grad() to get parameter for optimizer below

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    k=0
    loss, train_rmse, val_rmse, test_rmse = [], [], [], []
    train_wrmse, val_wrmse, test_wrmse = [], [], []
    for epoch in tqdm(range(n_epochs)):
        # Call train fuction here >> return loss
        loss += [train(train_data, model, optimizer, loss=weighted_mse_loss)]
        
        # Call test function here >> return RMSE loss
        train_wrmse += [test(train_data, model, metric=weighted_mse_loss)]
        train_rmse += [test(train_data, model, metric=F.mse_loss)]
        
        val_wrmse += [test(val_data, model, metric=weighted_mse_loss)]
        val_rmse += [test(val_data, model, metric=F.mse_loss)]
        
        test_wrmse += [test(test_data, model, metric=weighted_mse_loss)]
        test_rmse += [test(test_data, model, metric=F.mse_loss)]

        if epoch+1 %10==0:
            print(f'Epoch: {epoch+1:03d}, Loss: {loss[-1]:.4f}, Train: {train_rmse[-1]:.4f}, '
                  f'Val: {val_rmse[-1]:.4f}, Test: {test_rmse[-1]:.4f}')

        results = pd.DataFrame({
            'loss': loss,
            'train_rmse': train_rmse, 'val_rmse': val_rmse, 'test_rmse': test_rmse,
            'train_wrmse': train_wrmse, 'val_wrmse': val_wrmse, 'test_wrmse': test_wrmse,
            'time':(time.time()-t0)/60
        })
        
        ## Debugging
        #clear_output()
        '''
        print('\nloss: ', loss, 
              '\ntrain_rmse: ', train_rmse, 
              '\nval_rmse: ', val_rmse, 
              '\ntest_rmse: ', test_rmse,
              '\ntrain_wrmse: ', train_wrmse, 
              '\nval_wrmse: ', val_wrmse, 
              '\ntest_wrmse: ', test_wrmse,
              '\ntime: ', (time.time()-t0)/60)
        '''
        #visualize_loss(results, metric='wrmse').show()
        #print(results.to_string())

        # enable early stopping
        if (epoch > 1) and abs(loss[-1]/loss[-2]-1) < min_acc :
            k += 1
        if k> e_patience:
            print('Early stopping')
            break

    return results, model
"""

In [ ]:
'''
def visualize_loss(results, metric='rmse'):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=results.index, y=results['train_'+metric], name = 'train_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['val_'+metric], name = 'val_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['test_'+metric], name = 'test_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['loss'], name = 'loss'))

    fig.update_yaxes(title_text=metric.upper())
    fig.update_xaxes(title_text="Epoch")

    return fig
'''

In [ ]:
'''
N_EPOCHS = 300
E_PATIENCE = 50
LEARNING_RATE = 0.01

model_params = {"hidden_channels":32, 'conv':SAGEConv}

results, trained_model = train_test(
    Model, model_params, learning_rate=LEARNING_RATE, e_patience = E_PATIENCE, n_epochs=N_EPOCHS)
'''

In [ ]:
#visualize_loss(results, metric='wrmse')

In [ ]:
#trained_model.encoder(test_data.x_dict, test_data.edge_index_dict)['session']

In [ ]:
#trained_model.state_dict()['encoder.linear2.session.weight']

# Recommendation

In [ ]:
'''
@torch.no_grad()
def recommendation(user_id, model, x_dict, edge_index_dict):
  # Get model decoder
  #model = Model(**model_params)
  with torch.no_grad():
    encoder = model.encoder(data.x_dict, data.edge_index_dict)

  # Get node representations for users and movies
  user_representations = encoder['session']
  movie_representations = encoder['aid']

  # Compute the dot product between user and movie representations to get edge weights
  edge_weights = user_representations.mm(movie_representations.T)

  # Make predictions for each user by taking the top k largest edge weights
  k = 20  # number of recommendations to make
  _, top_k_indices = edge_weights.topk(k, dim=1)
  recommendations = top_k_indices.numpy()

  # Print recommendations for the first user
  print(f'Recommendations for user {user_id}: {recommendations[user_id]}')
'''

In [ ]:
#session = 1
#recommendation(session, model, Rtest_data.x_dict, Rtest_data.edge_index_dict)

In [ ]:
#session = 2
#recommendation(session, model, data.x_dict, data.edge_index_dict)

In [ ]:
#session = 999
#recommendation(session, model, data.x_dict, data.edge_index_dict)

# Submission

Submission File
For each `session` id and `type` combination in the test set, you must predict the `aid` values in the `label` column, which is space delimited. You can predict up to 20 `aid` values per row. The file should contain a header and have the following format:

```
session_type,labels
12906577_clicks,135193 129431 119318 ...
12906577_carts,135193 129431 119318 ...
12906577_orders,135193 129431 119318 ...
12906578_clicks, 135193 129431 119318 ...
etc.
```